# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd

In [2]:
# for challenge 2
from scipy.stats import ttest_1samp
#for challenge 3
from scipy.stats import t
#for challenge 4
from statsmodels.stats.proportion import proportions_ztest

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [11]:
# Run this code:

salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [12]:
# Your code here:
salaries.head(15)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
5,"ABBASI, CHRISTOPHER",STAFF ASST TO THE ALDERMAN,CITY COUNCIL,F,Salary,NaN,50436.0,NaN
6,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,46.10
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
8,"ABBATEMARCO, JAMES J",FIRE ENGINEER-EMT,FIRE,F,Salary,NaN,103350.0,NaN
9,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [13]:
# Your code here:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
Name                 33183 non-null object
Job Titles           33183 non-null object
Department           33183 non-null object
Full or Part-Time    33183 non-null object
Salary or Hourly     33183 non-null object
Typical Hours        8022 non-null float64
Annual Salary        25161 non-null float64
Hourly Rate          8022 non-null float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [14]:
# Your code here:
salaries['Salary or Hourly'][(salaries['Salary or Hourly'] == 'Hourly')].value_counts()
salaries['Salary or Hourly'][(salaries['Salary or Hourly'] == 'Salary')].value_counts()


Salary    25161
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [15]:
# Your code here:
salaries_gb = salaries.groupby('Department', as_index=False).agg({'Name':'count'})
salaries_gb = salaries_gb.rename(columns = {'Name':'Count'})
salaries_gb.head()

,Department,Count
0,ADMIN HEARNG,39
1,ANIMAL CONTRL,81
2,AVIATION,1629
3,BOARD OF ELECTION,107
4,BOARD OF ETHICS,8


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [16]:
# Your code here:
ttest_1samp(salaries['Hourly Rate'][(salaries['Salary or Hourly'] == 'Hourly')], 30)

Ttest_1sampResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92)

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [17]:
# Your code here:
ttest_1samp(salaries['Annual Salary'][(salaries['Department'] == 'POLICE') & 
                                      (salaries['Salary or Hourly'] == 'Salary')], 86000)

Ttest_1sampResult(statistic=3.081997005712994, pvalue=0.0020603403550965137)

Using the `crosstab` function, find the department that has the most hourly workers. 

In [18]:
# Your code here:
con_table = pd.crosstab(salaries['Department'], 
                        salaries['Salary or Hourly'] == 'Hourly')
con_table

Salary or Hourly,False,True
Department,,
ADMIN HEARNG,39,0
ANIMAL CONTRL,62,19
AVIATION,547,1082
BOARD OF ELECTION,107,0
BOARD OF ETHICS,8,0
BUDGET & MGMT,44,2
BUILDINGS,269,0
BUSINESS AFFAIRS,164,7
CITY CLERK,84,0


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [19]:
# Your code here:
ttest_1samp(salaries['Hourly Rate'][(salaries['Department'] == 'STREETS & SAN') & 
                                    (salaries['Salary or Hourly'] == 'Hourly')], 35)

Ttest_1sampResult(statistic=-9.567447887848152, pvalue=3.3378530564707717e-21)

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [20]:
# Your code here:
df = len(salaries['Hourly Rate'])-1
hourly_mean = salaries['Hourly Rate'].mean()
hourle_std = salaries['Hourly Rate'].std()

ci_hourly = t.interval(0.05,df, hourly_mean, hourle_std)
ci_hourly

(32.029011565590146, 33.54810386697242)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [21]:
# Your code here:
salaried_mean = salaries['Annual Salary'].mean()
salaried_std = salaries['Annual Salary'].std()

ci_salaried = t.interval(0.05,df, salaried_mean, salaried_std)
ci_salaried

(85467.5542594973, 88106.44532080577)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [22]:
# Your code here:
prop = salaries['Hourly Rate'][(salaries['Salary or Hourly'] == 'Hourly')].count()
ttl = salaries['Salary or Hourly'].count()

proportions_ztest(prop, ttl, 0.25)[1]

0.0004481127249057967